In this notebook, we are going to extract the segments included in polygons based on a percentage of included pixels.  

***Import necessary libraries***

In [7]:
import rioxarray as rxr
from rasterio.plot import show
import xarray as xr
import rasterio
import numpy as np
import rasterio as rio

Lets create the files variables

In [8]:
class_ID =   "D:/General/ExaplAInability_Data/transfer_6060512_files_e989f8bb/class_segment_polygon/class_ID_2020.tif"
segment_ID = 'D:/General/ExaplAInability_Data/transfer_6060512_files_e989f8bb/class_segment_polygon/segment_ID_2020.tif'
polygon_ID = 'D:/General/ExaplAInability_Data/transfer_6060512_files_e989f8bb/class_segment_polygon/polygon_ID.tif'

# Import the files

First lets create some usefull functions.

In [9]:
def open_tif(file, infos=False):
    with xr.open_rasterio(file) as ds:
        if infos:
            print(ds)
            print(ds.rio.crs)
            print(ds.rio.bounds())
            print(ds.rio.resolution())
            print(ds.rio.shape)
            print(ds.rio.transform())
            print(ds.rio.count)
    o_file = rxr.open_rasterio(file)
    return o_file

def show_file(o_file):
    display(o_file)
    show(o_file.data)
    return

Lets load the files we need.

In [10]:
class_ID_array = open_tif(class_ID)
segment_ID_array = open_tif(segment_ID)
polygon_ID_array = open_tif(polygon_ID)

C:\Users\antoi\AppData\Local\Temp\ipykernel_20364\3002180342.py:2: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  with xr.open_rasterio(file) as ds:
C:\Users\antoi\AppData\Local\Temp\ipykernel_20364\3002180342.py:2: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  with xr.open_rasterio(file) as ds:
C:\Users\antoi\AppData\Local\Temp\ipykernel_20364\3002180342.py:2: DeprecationWarning: open_rasterio is Deprecated in favor of rioxarray. For information about transitioning, see: https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html
  with xr.open_rasterio(file) as ds:


Lets make sure all the shapes are the same.

In [11]:
# print a small part of the array
print(class_ID_array.shape)
print(polygon_ID_array.shape)
print(segment_ID_array.shape)

class_ID_array_full = class_ID_array.data[0]
polygon_ID_array_full = polygon_ID_array.data[0]
segment_ID_array_full = segment_ID_array.data[0]

print(class_ID_array_full.shape)
print(polygon_ID_array_full.shape)
print(segment_ID_array_full.shape)

# show number of unique values
unique_class_ID = np.unique(class_ID_array_full)
unique_polygon_ID = np.unique(polygon_ID_array_full)
unique_segment_ID = np.unique(segment_ID_array_full)

print("Number of unique values in class_ID_array_full:", len(unique_class_ID))
print("Number of unique values in polygon_ID_array_full:", len(unique_polygon_ID))
print("Number of unique values in segment_ID_array_full:", len(unique_segment_ID))

(1, 4513, 5183)
(1, 4513, 5183)
(1, 4513, 5183)
(4513, 5183)
(4513, 5183)
(4513, 5183)
Number of unique values in class_ID_array_full: 9
Number of unique values in polygon_ID_array_full: 999
Number of unique values in segment_ID_array_full: 1447918


# Create test data

Skip this part to go to the results.  
Here we are creating some test data to test the function.  
The data follows the same structure as the real data.

In [12]:
# create test arrays
test_class_ID_array = np.random.randint(0, 8, (10, 10))
# polygons should be a matrix with groups of pixels with the same ID other is Nan
Nan = np.nan
test_polygon_ID_array = [[0, 0, 0, Nan, Nan, Nan, Nan, 1, 1, 1],
                   [0, 0, 0, Nan, Nan, Nan, Nan, 1, 1, 1],
                   [0, 0, 0, Nan, Nan, Nan, Nan, 1, 1, 1],
                   [Nan, Nan, Nan, Nan, Nan, Nan, Nan, Nan, Nan, Nan],
                   [Nan, Nan, Nan, 2, 2, 2, 2, 2, Nan, Nan],
                   [Nan, Nan, Nan, 2, 2, 2, 2, 2, Nan, Nan],
                   [Nan, Nan, Nan, 2, 2, 2, 2, 2, Nan, Nan],
                   [Nan, Nan, Nan, 2, 2, 2, 2, 2, Nan, Nan],
                   [Nan, Nan, Nan, Nan, Nan, Nan, Nan, Nan, Nan, Nan],
                   [Nan, Nan, Nan, Nan, Nan, Nan, Nan, Nan, Nan, Nan]]
test_polygon_ID_array = np.array(test_polygon_ID_array)

test_segment_ID_array = [[14, 14, 7, 7, Nan, Nan, Nan, 2, 2, Nan],
                   [9, 8, 8, 8, Nan, Nan, Nan, Nan, Nan, 3],
                   [9, 1, 1, Nan, Nan, Nan, Nan, Nan, Nan, 3],
                   [Nan, Nan, Nan, Nan, Nan, Nan, Nan, Nan, Nan, Nan],
                   [Nan, 10, Nan, 4, 4, 4, 11, Nan, Nan, Nan],
                   [Nan, 10, Nan, 4, Nan, 5, 5, Nan, Nan, Nan],
                   [Nan, Nan, Nan, 6, 6, 12, Nan, Nan, Nan, Nan],
                   [Nan, Nan, Nan, Nan, Nan, 12, Nan, Nan, Nan, Nan],
                   [Nan, Nan, Nan, Nan, Nan, 12, Nan, Nan, Nan, Nan],
                   [Nan, Nan, Nan, Nan, Nan, Nan, Nan, Nan, Nan, Nan]]
test_segment_ID_array = np.array(test_segment_ID_array)

# fill segment_ID nan values with 0
test_segment_ID_array = np.nan_to_num(test_segment_ID_array, nan=0)

print(test_class_ID_array)
print(test_polygon_ID_array)
print(test_segment_ID_array)
# show shape
print(test_class_ID_array.shape)
print(test_polygon_ID_array.shape)
print(test_segment_ID_array.shape)


[[0 7 1 7 7 0 5 5 0 1]
 [5 0 2 6 4 7 2 0 1 6]
 [2 3 3 7 1 4 5 4 1 4]
 [6 2 2 6 7 1 2 2 0 4]
 [1 6 0 6 6 1 2 3 6 7]
 [6 6 1 3 7 7 2 5 7 7]
 [2 4 7 0 2 0 4 7 6 0]
 [4 2 3 3 6 3 0 3 0 7]
 [2 6 7 5 5 4 2 0 3 6]
 [1 1 0 3 7 5 4 7 3 4]]
[[ 0.  0.  0. nan nan nan nan  1.  1.  1.]
 [ 0.  0.  0. nan nan nan nan  1.  1.  1.]
 [ 0.  0.  0. nan nan nan nan  1.  1.  1.]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan  2.  2.  2.  2.  2. nan nan]
 [nan nan nan  2.  2.  2.  2.  2. nan nan]
 [nan nan nan  2.  2.  2.  2.  2. nan nan]
 [nan nan nan  2.  2.  2.  2.  2. nan nan]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]]
[[14. 14.  7.  7.  0.  0.  0.  2.  2.  0.]
 [ 9.  8.  8.  8.  0.  0.  0.  0.  0.  3.]
 [ 9.  1.  1.  0.  0.  0.  0.  0.  0.  3.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. 10.  0.  4.  4.  4. 11.  0.  0.  0.]
 [ 0. 10.  0.  4.  0.  5.  5.  0.  0.  0.]
 [ 0.  0.  0.  6.  6. 12.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. 12.  0.  0.  0.

Show and analyze the test data.

In [13]:
print(test_class_ID_array.shape)
print(class_ID_array_full.shape)
print(test_polygon_ID_array.shape)
print(polygon_ID_array_full.shape)
print(test_segment_ID_array.shape)
print(segment_ID_array_full.shape)

print(test_class_ID_array[0,0])
print(class_ID_array_full[0,0])


# reduce the size of the arrays
start_x = 0
end_x = 2000
start_y = 0
end_y = 2000
class_ID_array_reduced = class_ID_array_full[:end_x, :end_y]
polygon_ID_array_reduced = polygon_ID_array_full[:end_x, :end_y]
segment_ID_array_reduced = segment_ID_array_full[:end_x, :end_y]

print(class_ID_array_reduced.shape)
print(polygon_ID_array_reduced.shape)
print(segment_ID_array_reduced.shape)

print(test_class_ID_array)
print(test_polygon_ID_array)
print(test_segment_ID_array)

(10, 10)
(4513, 5183)
(10, 10)
(4513, 5183)
(10, 10)
(4513, 5183)
0
nan
(2000, 2000)
(2000, 2000)
(2000, 2000)
[[0 7 1 7 7 0 5 5 0 1]
 [5 0 2 6 4 7 2 0 1 6]
 [2 3 3 7 1 4 5 4 1 4]
 [6 2 2 6 7 1 2 2 0 4]
 [1 6 0 6 6 1 2 3 6 7]
 [6 6 1 3 7 7 2 5 7 7]
 [2 4 7 0 2 0 4 7 6 0]
 [4 2 3 3 6 3 0 3 0 7]
 [2 6 7 5 5 4 2 0 3 6]
 [1 1 0 3 7 5 4 7 3 4]]
[[ 0.  0.  0. nan nan nan nan  1.  1.  1.]
 [ 0.  0.  0. nan nan nan nan  1.  1.  1.]
 [ 0.  0.  0. nan nan nan nan  1.  1.  1.]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan  2.  2.  2.  2.  2. nan nan]
 [nan nan nan  2.  2.  2.  2.  2. nan nan]
 [nan nan nan  2.  2.  2.  2.  2. nan nan]
 [nan nan nan  2.  2.  2.  2.  2. nan nan]
 [nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan]]
[[14. 14.  7.  7.  0.  0.  0.  2.  2.  0.]
 [ 9.  8.  8.  8.  0.  0.  0.  0.  0.  3.]
 [ 9.  1.  1.  0.  0.  0.  0.  0.  0.  3.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. 10.  0.  4.  4.  4. 11.  0.  0.  0.]
 [ 0. 10.  0

# Create the function

Here we create th function that will extract the segments from the data.  
Based on a parameter, the function should retrieve segments X% included in the polygons.  
By default this value is 100, which means fully contained segments, but we will use 50% for the rest of the project.  

In [14]:
def find_contained_segments(class_ID, segment_ID, polygon_ID, output_file, included_pixel_percentage=100):
    # Retrieve all polygons
    polygons = np.unique(polygon_ID)
    polygons = polygons[~np.isnan(polygons)]
    iterations = len(polygons)

    # Iterate through polygons
    for i in range(len(polygons)):
        print(f"Iteration {i + 1}/{iterations}")
        # Retrieve pixels of polygon
        polygon = polygons[i]
        polygon_pixels = np.where(polygon_ID == polygon)
        polygon_pixels = np.array(polygon_pixels).T

        # Retrieve all segments of polygon
        polygon_segments = []
        for pixel in polygon_pixels:
            polygon_segments.append(segment_ID[pixel[0], pixel[1]])
        polygon_segments = np.unique(polygon_segments)

        # For each segment, calculate the percentage of included pixels within the polygon
        for segment in polygon_segments:
            segment_pixels = np.where(segment_ID == segment)
            segment_pixels = np.array(segment_pixels).T
            total_pixels = len(segment_pixels)
            
            segment_pixel_polygon = np.where((segment_ID == segment) & (polygon_ID == polygon))
            segment_pixel_polygon = np.array(segment_pixel_polygon).T
            included_pixels = len(segment_pixel_polygon)

            # Calculate the percentage of included pixels
            percentage_included = (included_pixels / total_pixels) * 100

            if percentage_included >= included_pixel_percentage:
                # Retrieve the class of the segment
                class_id = class_ID[segment_pixel_polygon[0, 0], segment_pixel_polygon[0, 1]]

                # Generate the formatted pixel coordinates as lists
                pixel_coords = [list(pixel) for pixel in segment_pixels]

                # Write to the file in the desired format
                output_file.write(f"Segment ID: {segment}, Polygon ID: {polygon}, Class ID: {class_id}, Pixels: {pixel_coords}\n")


In [18]:
import time
segment_ID = 'D:/General/ExaplAInability_Data/transfer_6060512_files_e989f8bb/class_segment_polygon/segment_ID_2020.tif'
segment_ID_array = open_tif(segment_ID)
segment_ID_array_full = segment_ID_array.data[0]

In [28]:
def retrieve_all_segments(segment_ID, output_file):
    # Retrieve all segments
    all_segments = np.unique(segment_ID)
    iterations = len(all_segments)

    # Iterate through all segments to write to the output file
    for i, segment in enumerate(all_segments):
        print(f"Iteration {i + 1}/{iterations}")
        # Find pixels of segment
        segment_pixels = np.argwhere(segment_ID == segment)

        # Generate the formatted pixel coordinates as lists
        pixel_coords = [list(pixel) for pixel in segment_pixels]

        # Write to the file for each segment
        output_file.write(f"Segment ID: {segment}, Polygon ID: 0, Class ID: 0, Pixels: {pixel_coords}\n")


# Interesting test

### **Why using the transpose ?**

Lets see what happens if we don't use the transpose.

In [16]:
output_file_name = "data/test_function_02.txt"
with open(output_file_name, 'w') as output_file:
    find_contained_segments(test_class_ID_array, test_segment_ID_array, test_polygon_ID_array, output_file, included_pixel_percentage=50)

Iteration 1/3
Iteration 2/3
Iteration 3/3


As we can see on the results above using the **test data**, using `np.where` returns two arrays, one giving the **row index** and the other the **column index** for the pixels that are included in the **polygon**.  
Using `np.array` enables to have **two list**, one for **x** and one for **y** for the pixels.  
Since we want to have ***'couple'*** of pixels, we need to **transpose** the array to have the x and y coordinates for **each pixel**.  
Doing so we can easily iterate through each pixel and create the **segments**.

The same principle applies later on for the segments pixels.

# Extract 50% included segments

Now we can call the function to extract the segments.  
The segments are directly written to a file.  

In [ ]:
output_file_name = "data/contained_segments.txt"
with open(output_file_name, 'w') as output_file:
    # run the function on the full arrays
    find_contained_segments(class_ID_array_full, segment_ID_array_full, polygon_ID_array_full, output_file, included_pixel_percentage=50)
    
    # to test the function on reduced arrays
    #find_fully_contained_segments(class_ID_array_reduced, segment_ID_array_reduced, polygon_ID_array_reduced, output_file)
    
    # to test the function on test arrays
    #find_fully_contained_segments(test_class_ID_array, test_segment_ID_array, test_polygon_ID_array, output_file)

# Extract all segments

We can use the second function to extract all segments in the same format as the first function.

In [29]:
output_file_name = "data/contained_all_segments.txt"
with open(output_file_name, 'w') as output_file:
    # run the function on the full arrays
    retrieve_all_segments(segment_ID_array_full, output_file)#segment_ID_array_full

Iteration 1/14
Iteration 2/14
Iteration 3/14
Iteration 4/14
Iteration 5/14
Iteration 6/14
Iteration 7/14
Iteration 8/14
Iteration 9/14
Iteration 10/14
Iteration 11/14
Iteration 12/14
Iteration 13/14
Iteration 14/14


# Conclusion

We used the function to extract the segments from the data for 0.001% (if one pixel of a segment is included in a polygon then all the segment is retrieve), 50% and 100%.  
In the next notebook, we are using the files to gain insights and make a decision on the percentage to use.